In [1]:
# imports
import snntorch as snn
from snntorch import surrogate
from snntorch import backprop
from snntorch import functional as SF
from snntorch import utils
from snntorch import spikeplot as splt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import torch.nn.functional as F

import matplotlib.pyplot as plt
import numpy as np
import itertools

In [5]:
batch_size = 128
data_path='/home/arsalikhov/Documents/PSYCH420_final_project/cifar'
subset=10

dtype = torch.float
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")

In [10]:
transform = transforms.Compose([
            transforms.ToTensor(),
            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

cifar_train = datasets.CIFAR10(data_path, train=True, download=False, transform=transform)
cifar_test = datasets.CIFAR10(data_path, train=False, download=False, transform=transform)


# Create DataLoaders
train_loader = DataLoader(cifar_train, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)
test_loader = DataLoader(cifar_test, batch_size=batch_size, shuffle=True, drop_last=True, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


In [11]:
# neuron and simulation parameters
spike_grad = surrogate.fast_sigmoid(slope=25)
beta = 0.5
num_steps = 50

In [16]:
net = nn.Sequential(nn.Conv2d(3, 64, 5),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    nn.Conv2d(32, 64, 5),
                    nn.MaxPool2d(2),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True),
                    F.adaptive_avg_pool2d((1,1))
                    nn.Flatten(),
                    nn.Linear(512, 10),
                    snn.Leaky(beta=beta, spike_grad=spike_grad, init_hidden=True, output=True)
                    ).to(device)

In [17]:
data, targets = next(iter(train_loader))
data = data.to(device)
targets = targets.to(device)

for step in range(num_steps):
    spk_out, mem_out = net(data)

RuntimeError: mat1 and mat2 shapes cannot be multiplied (128x1600 and 1024x10)